In [32]:
import cv2

In [117]:
img_ok = cv2.imread('../team/images/true_ok/GSY827AN7A1356_AAO11960K_PKT10_CM1EQSUA0012_20220711210457_DirectLight_OK.jpg')
img_ng = cv2.imread('../team/images/true_ng/GSY827AN7B0519_AAO12705K_PKT08_CM1EQSUA0011_20220711213213_DirectLight_NG.jpg')
img_ng2 = cv2.imread('../team/images/true_ng/GSY827AN7A1641_AAO19043K_PKT13_CM1EQSUA0012_20220711220653_DirectLight_NG.jpg')
img_over = cv2.imread('../team/images/overkill/GSY827AN7B0355_AAO04537K_PKT01_CM1EQSUA0011_20220711234650_DirectLight_NG.jpg')


In [118]:
def img_resize(img, resize_size = 1600):
    width_origin = img.shape[1]
    height_origin = img.shape[0]
    hw_list = [height_origin, width_origin]

    img_ratio = float(resize_size)/ max(hw_list)
    # print(img_ratio)
    max_index = hw_list.index(max(hw_list))
    if max_index ==0:
        img = cv2.resize(img, (int(width_origin * img_ratio), resize_size),interpolation=cv2.INTER_CUBIC)
    else:
        img = cv2.resize(img, (resize_size,int(height_origin * img_ratio)),interpolation=cv2.INTER_CUBIC)

    return img


In [132]:
def find_large_label(img, img_bin, show_img = False):
    test_img = img.copy()
    _, labels, stats, centroids = cv2.connectedComponentsWithStats(img_bin,)
    stats = sorted(stats, key = lambda x: x[4],reverse=True)
    if show_img:
        for i, rec in enumerate(stats[:2]):
            x,y,w,h, area = rec
            cv2.rectangle(test_img, (x,y,w,h), (0,0,255),thickness=8)

        cv2.imshow('items', img_resize(test_img, 600))
        cv2.imshow('items_bin', img_resize(img_bin, 600))
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        # print(stats[0])
    if stats[0][1]==0:
        return stats[1]
    return stats[0]

In [133]:
def get_threshold(img,gray,bin_inverse = True):
    if bin_inverse:
        _, img_bin = cv2.threshold(gray, -1,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    else:
        _, img_bin = cv2.threshold(gray, -1,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    return img_bin


In [134]:
# img_ng = cv2.imread('../team/images/true_ng/GSY827AN7D2341_AAO08637K_PKT15_CM1EQSUA0012_20220714014226_DirectLight_NG.jpg')
# # gray = cv2.cvtColor(img_ng, cv2.COLOR_BGR2GRAY)
# # img_bin = get_threshold(img_ng,gray,bin_inverse = True)
# # find_large_label(img_ng, img_bin, show_img = True)
# item_img, item_gray, item_bin = preproces(img_ng)

# cv2.imshow('img', img_resize(img_ng, 600))
# cv2.imshow('items', img_resize(item_img, 600))
# cv2.waitKey(0)
# cv2.destroyAllWindows()



In [135]:
def preproces(img, bin_inverse = True, only_bin = False):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_bin = get_threshold(img,gray,bin_inverse = True)

    large_stat = find_large_label(img, img_bin)
    print(large_stat)
    try:
        item_img = img[large_stat[1]:large_stat[1]+large_stat[3],large_stat[0]:large_stat[0]+large_stat[2], :].copy()
        item_gray = gray[large_stat[1]:large_stat[1]+large_stat[3],large_stat[0]:large_stat[0]+large_stat[2]].copy()
    except:
        item_img, item_gray= img,gray

    item_bin = get_threshold(item_img,item_gray,bin_inverse = False)
    return  item_img, item_gray, item_bin

In [136]:
import numpy as np
def img_mask(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(img_hsv,(0,0,0),(0,0,100))
    imask = mask>0
    col = np.zeros_like(img_hsv, np.uint8)
    col[imask] = img_hsv[imask]
    col = cv2.cvtColor(col,cv2.COLOR_HSV2BGR)
    return col

In [137]:
import math
# 두 점의 각도 계산
def get_angle(pt1, pt2):
    xd = abs(pt2[0] - pt1[0])
    yd = abs(pt2[1] - pt1[1])
    if xd == 0:
        radian = 0.5*math.pi
    else:
        radian = math.atan(yd/xd)
    return radian


In [138]:
get_angle([0,0], [1,-(3**0.5)])*180/math.pi

59.99999999999999

In [139]:
# 각도를 계산할 두 꼭짓점 찾기
def get_points(cnt):
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)   # 중심점과 각도를 4개의 꼭지점 좌표로 변환
    box = np.int0(box)          # 정수로 변환
    # pt1_ind = cnt[:,:,1].argmin()
    # pt2_ind = cnt[:,:,0].argmin()

    return box

In [140]:
math.sin(1/6*math.pi)

0.49999999999999994

In [157]:
def cnt_test(cnt):
    volum_ratio_bound = 0.005
    """_summary_

    Args:
        cnt (_type_): _description_

    Returns:
        _type_: _description_
    """
    # w_max, w_min, h_max, h_min = (
    #     cnt[:, :, 0].max(),
    #     cnt[:, :, 0].min(),
    #     cnt[:, :, 1].max(),
    #     cnt[:, :, 1].min(),
    # )
    # print(w_max - w_min, h_max - h_min)
    area = cv2.contourArea(cnt)
    arcLen = cv2.arcLength(cnt, closed=True)
    box = get_points(cnt)
    s_box = sorted(box, key = lambda x: x[0]+x[1])
    # print(s_box)
    point1, point2 = s_box[0],s_box[2]
    angle = get_angle(point1, point2)
    # print(max_area, area,round(area/max_area,3), arcLen)
    sin, cos = math.sin(angle), math.cos(angle)
    cal_max_box = ((arcLen**2)/4 - 2*area)*(sin*cos)+area
    # max_box = (w_max-w_min)*(h_max-h_min)
    max_box = cv2.contourArea(box)
    print(round(cal_max_box/max_box,3), angle*180/math.pi)
    # print(box[1:3], angle*180/math.pi)
    if  cal_max_box/max_box < 1-volum_ratio_bound or cal_max_box/max_box > 1+volum_ratio_bound :
        pred = 'NG'
    else:
        pred = 'ok'
    return point1, point2, pred

In [142]:
def check_contain(img_shape, cnt):
    center_bound = 100
    (x,y), radius = cv2.minEnclosingCircle(cnt)
    # center = (int(x), int(y))
    radius = int(radius)
    center_diff = ((int(img_shape[0]/2)-int(y))**2+(int(img_shape[1]/2) -int(x))**2)**0.5
    # print(center, (img_shape[1]/2,img_shape[0]/2), center_diff)
    if center_diff < center_bound:
        return True
    return False

In [143]:
def find_contours(img, ip = None):
    # img = cv2.resize(img, (0,0), fx = 0.3, fy = 0.3, interpolation=cv2.INTER_CUBIC)
    item_img, item_gray, item_bin = preproces(img)
    col = img_mask(item_img)
    # contour, hierachy = cv2.findContours(item_bin, cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    contour, hierachy = cv2.findContours(item_bin, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    result = False
    for i, cnt in enumerate(contour):

        if cv2.contourArea(cnt) > 1500000:
            if check_contain(item_img.shape, cnt):
                cv2.drawContours(item_img, [cnt], 0,(255,0,0),5)
                result = True
                break
        

    if ip:
        cv2.putText(img, ip, (10,60),cv2.FONT_HERSHEY_SIMPLEX,3,(0,255,0),3)
    
    pred = 'NG'
    if result:
        point1, point2, pred = cnt_test(cnt)
        # print(box)
        j=1
        cols = [(255,255,255),(0,0,255),(0,255,0),(255,0,0)]
        # point1x,point1y = point1
        # point2x,point2y = point2
        
        cv2.circle(item_img,point1,5,cols[1],10) 
        cv2.circle(item_img,point2,5,cols[2],10) 

        
    cv2.putText(item_img, 'predicted '+pred, (10,60),cv2.FONT_HERSHEY_SIMPLEX,3,(0,255,0),3)


    # print(item_img.shape)
    # cv2.imshow('img', img)
    cv2.imshow('item_img', cv2.resize(item_img, (0,0), fx = 0.3, fy = 0.3, interpolation=cv2.INTER_CUBIC))
    # cv2.imshow('col', col)
    # cv2.imshow('bin', item_bin)
    
    key_val = cv2.waitKey(0)
    cv2.destroyAllWindows()

    return key_val



In [144]:
find_contours(img_ng, ip = 'NG')
find_contours(img_ng2, ip = 'NG')
find_contours(img_ok, ip = 'ok')
find_contours(img_over, ip = 'ok')

[      0     140    2464    1772 1667785]
[      0     144    2464    1762 1831883]
[      0     106    2464    1794 1745382]
[      0     129    2464    1766 1664577]


13

In [148]:
import os
import random
import time
def check_img(kind = 'overkill'):
    random.seed(time.time_ns()%10000)
    if kind == 'all':
        paths = os.listdir('../team/images/')
        img_paths = []
        for p in paths:
            file_path = '../team/images/'+p+'/'
            img_paths = list(map(lambda x: [file_path+ x, p], os.listdir(file_path)))
        while True:
            ip = random.choice(img_paths)
            img = cv2.imread(ip[0])
            
            key_val= find_contours(img, ip =ip[1])
            
            if key_val == 113:
                break

    else:
        file_path = '../team/images/'+kind+'/'
        img_paths = os.listdir(file_path)
        for i in range(len(img_paths)):
            img = cv2.imread(file_path+img_paths[i])
            key_val = find_contours(img,  ip = kind)
            if key_val == 113: # ord('q')
                break

In [156]:
check_img('true_ng')

[     40     144    2319    1765 1585512]
0.968 0.23930079028303866
[      0     144    2464    1769 1685854]
0.95 0.10105065274517852
[      0     161    2464    1769 1834001]
1.025 0.43764225508377574
[      0     144    2464    1762 1831883]
0.982 0.10913468613578126
[      0     153    2464    1762 2842876]
0.951 0.0
[      0     158    2464    1770 2753339]
0.951 0.0
[      0     130    2464    1797 1744433]
1.001 0.06837201947086552
[      0     123    2464    1761 1647404]
0.96 0.0
[      0     144    2464    1797 1684902]
0.957 0.19894287936062627
[      0     140    2464    1772 1667785]
0.998 0.0688650864458102
[      0     117    2464    1770 1692319]
0.974 0.1368254820237406
[      0     129    2464    1806 1725318]
0.983 0.06837201947086552
[      0     155    2416    1792 1645204]
0.971 0.06837201947086552
[      0     122    2464    1797 1728219]
0.975 0.2683623443873789
[      0     103    2464    1789 1753556]
0.992 0.16871499974021562
[      0     161    2464    1784 

In [155]:
check_img('overkill')

[      0     134    2464    1800 1751696]
1.0 0.06837201947086552
[      0     129    2464    1766 1664577]
0.998 0.13764725353806873
[      0     150    2464    1762 1656273]
1.003 0.2065944634734015
[      0      99    2464    1802 1767679]
1.013 0.4104702039341763
[      0     137    2464    1811 1756314]
1.008 0.27348613066685795
[      0     139    2464    1805 1715774]
0.998 0.0
[      0     137    2464    1768 1698671]
1.001 0.0688650864458102
[      0     151    2464    1775 1683656]
1.002 0.10255796835506453
[      0     145    2464    1780 1700053]
1.006 0.20511527952129663
[      0     137    2464    1811 1756556]
1.0 0.06837201947086552


In [158]:
check_img('true_ok')

[      0     106    2464    1794 1745382]
1.0 0.06837201947086552
[      0     141    2464    1772 1696337]
1.0 0.06837201947086552
[      0     135    2464    1755 1644439]
0.999 0.0
[      0     131    2464    1756 1631219]
1.002 0.1026191968619867
[      0     131    2464    1799 1731900]
1.009 0.27364940348652206
[      0     147    2464    1774 1673518]
0.999 0.0
[      0     126    2464    1817 1675165]
0.999 0.0
[      0     135    2464    1809 1776288]
0.999 0.0
[      0     107    2464    1795 1763492]
1.0 0.0
